In [1]:
import os
import sys
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline

<h1>Definition of functions</h1>

In [143]:
def read_SRIM(file):
    start = 0
    lst = []
    with open(file) as f:
        lines = f.readlines()
        for idx, line in enumerate(lines):
            if ('-----------------------------------------------------------' in line):
                break
            if start == 1:
                line = line.replace(',', '.')
                words = line.split()
                LET_el = float(words[2])
                LET_nuc = float(words[3])
                energy = float(words[0])
                if words[1] == 'MeV':
                    energy *= 1_000.0
                if words[1] == 'GeV':
                    energy *= 1_000_000.0
                lst.append((energy, LET_el, LET_nuc))
            if ('--------------' in line):
                start = 1
    df = pd.DataFrame(lst, columns=['energy', 'LET_el', 'LET_nuc'])
    df['LET_el'] = df['LET_el'] * 0.001 # keV/nm
    df['LET_nuc'] = df['LET_nuc'] * 0.001 # keV/nm
    #df['LET'] = df['LET_el'] + df['LET_nuc'] # keV/nm
    #print(df)
    return df
    
def interpolate_LET(x, y):
    f = CubicSpline(x, y, bc_type='natural')
    return f

def energy_loss(LET, distance):
    return LET * distance

def calculate_filter(df, original_energy, binary_filter):
    binary_filter_nm = binary_filter * 1E+6
    thickness = int(np.max(binary_filter_nm))
    
    granularity = 10 # nm
    len_arr = int((int(thickness + 0.5) / granularity) + 1)
    
    x = np.linspace(0, thickness, len_arr)
    LET_func_el = interpolate_LET(df['energy'], df['LET_el'])
    LET_func_nuc = interpolate_LET(df['energy'], df['LET_nuc'])
    
    ion_energy = original_energy
    original_distance = 0
    el_energy, nuc_energy = 0, 0
    for idx, distance in enumerate(x):
        delta = distance - original_distance
        original_distance = distance
        LET_el = LET_func_el(ion_energy)
        LET_nuc = LET_func_nuc(ion_energy)
        energy_loss_el = energy_loss(LET_el, delta)
        energy_loss_nuc = energy_loss(LET_nuc, delta)
        el_energy += energy_loss_el
        nuc_energy += energy_loss_nuc
        ion_energy = ion_energy - energy_loss_el - energy_loss_nuc
        if distance in binary_filter_nm:
            result = [distance * 1E-6, 1000 * (LET_el + LET_nuc), ion_energy, el_energy, nuc_energy]
            print_results(result)
            #print(distance * 1E-7, 1000 * (LET_el + LET_nuc), ion_energy, el_energy, nuc_energy)
        else:
            #print(distance, binary_filter_nm)
            k=0
    return result

def print_results(res):
    print('------------------------------------------------------------------------------------------')
    print('Distance traveled in binary filter: ', round(res[0], 3), ' mm')
    print('Energy of the ion behind the binary filter: ', round(res[2] * 0.001, 3), ' MeV')
    print('LET in water of the ion behind the binary filter: ', round(res[1], 3), ' keV/um')
    print('Deposited energy in binary filter (elec. component): ', round(res[3] * 0.001, 3), ' MeV')
    print('Deposited energy in binary filter (nuclear component): ', round(res[4] * 0.001, 3), ' MeV')
    print('------------------------------------------------------------------------------------------')


<h1>Run ion through binary filter</h1>

In [145]:
original_energy = 572_836 # keV
binary_filter = np.array([1, 10, 100.43, 141.54]) # mm equivalent
file = '/home/sommema4/git/SRIM/He_in_Water_Liquid.txt'

df = read_SRIM(file)

print('The original ion energy: ', original_energy * 0.001, ' MeV')
res = calculate_filter(df, original_energy, binary_filter)

The original ion energy:  572.836  MeV
------------------------------------------------------------------------------------------
Distance traveled in binary filter:  1.0  mm
Energy of the ion behind the binary filter:  570.567  MeV
LET in water of the ion behind the binary filter:  2.272  keV/um
Deposited energy in binary filter (elec. component):  2.268  MeV
Deposited energy in binary filter (nuclear component):  0.001  MeV
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Distance traveled in binary filter:  10.0  mm
Energy of the ion behind the binary filter:  549.85  MeV
LET in water of the ion behind the binary filter:  2.332  keV/um
Deposited energy in binary filter (elec. component):  22.977  MeV
Deposited energy in binary filter (nuclear component):  0.01  MeV
----------------------------------------------------------------------------------------